In [2]:
## This notebook is to repeatedly subsample a vcf file to create many replicate, subsampled Site Frequency Spectra 
## Imports
import os
import easySFS
import ipyparallel as ipp

In [ ]:
## Start Ipcluster
## -------------------------------------------------------------------------------------------
## run the below line in terminal, or with '!' in jupyter notebooks
## !ipcluster start -n 10 --cluster-id="easySFS" --daemonize
## this will stop the ipcluster
## !ipcluster stop --cluster-id="easySFS" 

## link the cluster with ipyclient
ipyclient = ipp.Client(cluster_id="easySFS")

In [ ]:
## check to make sure the client is working by printing cores ready to go
print(len(ipyclient))
ipyclient.ids

In [2]:
## This function is used to subsample SNPS from a VCF file repeatedly
##     for a set number of individuals from each popoulation; only works with 2 populations

def subsampleSFS(infile, popfile, sumfile, reps=1, pop1size=10, pop2size=10):
            
        summaryOutputfile = open(sumfile, "w+")
        summaryOutputfile.write(str(reps)+" observations\n2\t"+str(pop1size)+"\t"+str(pop2size)+"\n")
        
        for i in range(1, reps+1, 1):
            
            #print("Subsampling the SFS number "+str(i))
            out = "output_"+str(i)
            ## create the easySFS.py command to run in the terminal
            cmd = "./easySFS.py -i "+ infile+ " -p "+popfile+" --proj="+str(pop1size)+","+str(pop2size)+" -o "+out
            print(cmd)
            #os.system(cmd)
        
        print("Finished subsampling...now compiling results")
        
        for i in range(1, reps+1, 1):
            print("compiling SFS "+str(i))
            
            ## locate the output file, open, combine all of the lines and add it to the output file
            jSFSfileName = infile.split(".")[0]+"_jointMAFpop1_0.obs"
            outFolder = "./output_"+str(i)
            pathtojSFS_File = os.path.join(outFolder, "fastsimcoal2", jSFSfileName)
            print(pathtojSFS_File)
            jSFSfile = open(pathtojSFS_File, "r")

            ##get ride of first two lines
            allLines = jSFSfile.readlines()
            allLines = allLines[2:]
            #print(allLines)

            for line in allLines:
                rows = line.split("\t")[1].strip()
                #print(rows)
                sites = rows.split(" ")
                for singleSiteFreq in sites:
                    #print(singleSiteFreq+"\t")
                    summaryOutputfile.write(singleSiteFreq+"\t")
 
            ## add return for each SFS flattened
            summaryOutputfile.write("\n")
        
        ## at the very end, delete the output directories
        rmcmd = "rm -rf output_*"
        os.system(rmcmd)

In [7]:
def subsampleSFS(infile, popfile, sumfile, reps=1, pop1size=10, pop2size=10):
            
        #summaryOutputfile = open(sumfile, "w+")
        #summaryOutputfile.write(str(reps)+" observations\n2\t"+str(pop1size)+"\t"+str(pop2size)+"\n")
        
        for i in range(1, reps+1, 1):
            
            #print("Subsampling the SFS number "+str(i))
            out = "output_"+str(i)
            ## create the easySFS.py command to run in the terminal
            cmd = "./easySFS.py -i "+ infile+ " -p "+popfile+" --proj="+str(pop1size)+","+str(pop2size)+" -a -o "+out
            print(cmd)
            os.system(cmd)
        
        print("Finished subsampling...now compiling results")
        
        for i in range(1, reps+1, 1):
            print("compiling SFS "+str(i))
            
            ## locate the output file, open, combine all of the lines and add it to the output file
            jSFSfileName = infile.split(".")[0]+"_jointMAFpop1_0.obs"
            outFolder = "./output_"+str(i)
            pathtojSFS_File = os.path.join(outFolder, "fastsimcoal2", jSFSfileName)
            print(pathtojSFS_File)
            
            cmd = "cp "+ pathtojSFS_File + " NonParBootstrapjSFS/"+str(i)+jSFSfileName
            print(cmd)
            os.system(cmd)
            
            '''
            jSFSfile = open(pathtojSFS_File, "r")

            ##get ride of first two lines
            allLines = jSFSfile.readlines()
            allLines = allLines[2:]
            #print(allLines)

            for line in allLines:
                rows = line.split("\t")[1].strip()
                #print(rows)
                sites = rows.split(" ")
                for singleSiteFreq in sites:
                    #print(singleSiteFreq+"\t")
                    summaryOutputfile.write(singleSiteFreq+"\t")
 
            ## add return for each SFS flattened
            summaryOutputfile.write("\n")
            '''
        ## at the very end, delete the output directories
        #rmcmd = "rm -rf output_*"
        #os.system(rmcmd)
  

In [8]:
##first one that i ran, medium about of SNPS ~ 2000
subsampleSFS("mergedThuja_minsamp4.vcf","popFile.txt", "SumOutTest.txt", reps=2, pop1size=80, pop2size=70)

./easySFS.py -i mergedThuja_minsamp4.vcf -p popFile.txt --proj=80,70 -a -o output_1
./easySFS.py -i mergedThuja_minsamp4.vcf -p popFile.txt --proj=80,70 -a -o output_2
Finished subsampling...now compiling results
compiling SFS 1
./output_1/fastsimcoal2/mergedThuja_minsamp4_jointMAFpop1_0.obs
cp ./output_1/fastsimcoal2/mergedThuja_minsamp4_jointMAFpop1_0.obs NonParBootstrapjSFS/1mergedThuja_minsamp4_jointMAFpop1_0.obs
compiling SFS 2
./output_2/fastsimcoal2/mergedThuja_minsamp4_jointMAFpop1_0.obs
cp ./output_2/fastsimcoal2/mergedThuja_minsamp4_jointMAFpop1_0.obs NonParBootstrapjSFS/2mergedThuja_minsamp4_jointMAFpop1_0.obs


In [ ]:
## second one i ran for a bunch of snps, ~ 12,000
subsampleSFS("mergedThuja_minsamp4.vcf","popFile.txt", "SumOutTest.txt", reps=100, pop1size=17, pop2size=14)

In [117]:
!cat ./output/fastsimcoal2/mergedThuja_minsamp4_jointMAFpop1_0.obs

1 observation
	d0_0	d0_1	d0_2	d0_3	d0_4	d0_5	d0_6	d0_7	d0_8	d0_9	d0_10	d0_11	d0_12	d0_13	d0_14	d0_15	d0_16	d0_17	d0_18	d0_19	d0_20	d0_21	d0_22	d0_23	d0_24	d0_25	d0_26	d0_27	d0_28	d0_29	d0_30	d0_31	d0_32	d0_33	d0_34	d0_35	d0_36	d0_37	d0_38	d0_39	d0_40	d0_41	d0_42
d1_0	175.8318466538853 123.6000671180889 30.9007303572111 10.55626694748666 4.245170062945142 1.580194344845177 1.174486558528347 1.465638137074886 0.7970046525011555 0.9521390927926832 1.032871048356148 0.7650261389020702 0.5062138993563078 0.2935458759128932 0.2059064111056363 0.1776481906685284 0.1425149831990507 0.09646851951252869 0.05517747496248704 0.02774115078199045 0.01450429660320834 0.009666477615132601 0.007495860550470934 0.004453092192715414 0.001433350784119682 0.0002146265231054115 2.061047190795328e-05 5.52398212924999e-06 1.181297809120913e-06 1.977144931770697e-07 5.136886029842225e-05 0.0004769984979595147 0.001885624887335356 0.004190277104022705 0.005823252721747591 0.005324116773090365 0.7464072396647548

In [53]:
!cat output/fastsimcoal2/mergedThuja_minsamp4_jointMAFpop1_0.obs

1 observation
	d0_0	d0_1	d0_2	d0_3	d0_4	d0_5	d0_6	d0_7	d0_8	d0_9	d0_10	d0_11	d0_12	d0_13	d0_14	d0_15	d0_16	d0_17	d0_18	d0_19	d0_20	d0_21	d0_22	d0_23	d0_24	d0_25	d0_26	d0_27	d0_28	d0_29	d0_30	d0_31	d0_32	d0_33	d0_34	d0_35	d0_36	d0_37	d0_38	d0_39	d0_40	d0_41	d0_42
d1_0	177.5141750117649 125.2783047207694 28.68104945054582 10.07420185470827 5.420534666462655 2.659185290108683 1.360534948457438 0.6009474296833608 0.5589506945243901 0.5386241597705784 0.453429805425867 0.1865046171193847 0.08849102214970152 0.03428487230634661 0.01104414932668359 0.003059670072003078 0.0008015516073496019 0.03699552925437388 0.2040775210465372 0.3867608694151946 0.2983251329702055 0.08281304225225858 0.006505618114915021 0.004238650956768853 0.001400233267933154 0.0002351098776740768 0.0001412933833940959 0.0003401830465040701 0.000583874227581635 0.0006753180326042807 0.000587714451192482 0.0007735951382381874 0.002000324369326411 0.004221172587387721 0.005828977473077012 0.005324830508321048 0.00327833343

In [3]:
## Test runs of easySFS.py
## the -f flag forces a rewrite over the old files
!./easySFS.py -i MinSamp4/mergedThuja_minsamp4.vcf -p popFile.txt --proj=42,42 -f

usage: easySFS.py [-h] [-a] -i VCF_NAME -p POPULATIONS [--proj PROJECTIONS]
                  [--preview] [-o OUTDIR] [--ploidy PLOIDY] [--prefix PREFIX]
                  [--unfolded] [--dtype DTYPE] [--GQ GQUAL] [-f] [-v]
easySFS.py: error: argument -v: ignored explicit argument 'False'
